# word2vec

Считаем вектора слов

Суммируем все вектора каждого из текстов, сравниваем между собой

Суммируем все вектора абзацев, сравниваем послежовательно каждый абзац каждого текста, сравниаем близость по медиане

In [14]:
from gensim.models.keyedvectors import FastTextKeyedVectors

import numpy as np
from collections import OrderedDict

from scipy.spatial import distance
import operator

import statistics

In [ ]:
#используем коллекцию предобученных векторов слов
fasttext = FastTextKeyedVectors.load("araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model")

In [34]:
#проверяем что все корректно подгрузилось
fasttext['уважаемый'] 

C:\Users\Admin\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ -7.62082413e-02,   1.78887974e-02,  -2.55130790e-02,
         2.06156522e-02,  -2.61091962e-02,   1.12581283e-01,
        -4.76743840e-02,   4.94216681e-02,  -4.33580689e-02,
        -1.23744784e-02,  -8.83364230e-02,  -3.59828062e-02,
        -4.24147323e-02,  -5.06249294e-02,   5.97045124e-02,
        -2.56801471e-02,  -3.00195757e-02,  -3.64938304e-02,
         4.63778824e-02,  -8.25238228e-02,   1.79971680e-02,
        -2.25572549e-02,  -8.69684294e-02,   4.85437512e-02,
         4.35988493e-02,  -8.77935216e-02,   1.27777234e-01,
        -3.75761390e-02,  -8.42803344e-02,  -1.64243355e-02,
        -2.13620923e-02,  -6.77805543e-02,  -4.72699478e-02,
        -1.73333585e-02,   5.64810522e-02,  -5.13666123e-02,
        -6.55516423e-03,  -3.65715362e-02,  -9.69852433e-02,
         1.40067218e-02,   1.65425078e-03,   5.25458716e-02,
         1.52026797e-02,  -1.95472371e-02,   1.18016839e-01,
        -3.33277285e-02,   2.67862696e-02,   2.63341907e-02,
         7.68206567e-02,

In [3]:
def tokenize_and_vectorise_file(file):
    """переводим файл в набор листов, каждый элемент которых соответствует вектору слова"""
    tokenized_text = []
    words_not_found = 0
    with open (file, "r", encoding = "utf-8") as file:
        for line in file:
            tokenized_line = line.split(' ')
            tokenized_line = [tok.replace('\n', '')  for tok in tokenized_line if (len(tok) > 2)]
            if(tokenized_line):
                vectors_list = []
                for word in tokenized_line:
                    try:
                        vector = fasttext[word]
                        vectors_list.append(vector)
                    except:
                        words_not_found += 1
                if(vectors_list):
                    tokenized_text.append(vectors_list)
    print("words_not_found", words_not_found)
    return tokenized_text

In [4]:
putin = tokenize_and_vectorise_file('speech_putin_processed.txt')
rzn = tokenize_and_vectorise_file('speech_rzn_processed.txt')
tmn = tokenize_and_vectorise_file('speech_tmn_processed.txt')
tula = tokenize_and_vectorise_file('speech_tula_processed.txt')
yml = tokenize_and_vectorise_file('speech_yml_processed.txt')

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


words_not_found 29
words_not_found 52
words_not_found 2
words_not_found 46
words_not_found 22


In [5]:
def sum_text_vectores_by_word(text):
    """суммируем все вектора слов текста"""
    vector = np.zeros(len(text[0][0]))
    for line in text:
        for word_vec in line:
            vector = np.add(vector,word_vec)
    return vector

In [6]:
putin_vector = sum_text_vectores_by_word(putin)
rzn_vector = sum_text_vectores_by_word(rzn)
tmn_vector = sum_text_vectores_by_word(tmn)
tula_vector = sum_text_vectores_by_word(tula)
yml_vector = sum_text_vectores_by_word(yml)

In [8]:
#Попарно без повторений находим косинусную близость каждого из полученных векторов. Полчаем выводы

In [9]:
vectors_dict = {'ryazan':rzn_vector,"tyumen":tmn_vector,"tula": tula_vector, "yamal": yml_vector, "putin":putin_vector}
putin_cosine_distance_dict = {'ryazan':0,"tyumen":0,"tula": 0, "yamal": 0}
ordered_vectors_dict = OrderedDict(vectors_dict)
keys = list(ordered_vectors_dict.keys())
values = list(ordered_vectors_dict.values())

for dict_ind in range(len(keys)):
    current_vect = values[dict_ind]
    for compare_ind in range(dict_ind + 1,len(keys)):
        print(keys[dict_ind], "vs", keys[compare_ind])
        compared_vect = values[compare_ind]
        print(distance.cosine(current_vect,compared_vect))
        
        if(keys[compare_ind] == 'putin'):
            putin_cosine_distance_dict[keys[dict_ind]] = distance.cosine(current_vect,compared_vect)
    print()
sorted_x = sorted(putin_cosine_distance_dict.items(), key=operator.itemgetter(1))
print(sorted_x[-1][0],"is most similar to Putin speech")

ryazan vs tyumen
0.07299630528692735
ryazan vs tula
0.01810150656670917
ryazan vs yamal
0.025649192890273675
ryazan vs putin
0.10347123788346713

tyumen vs tula
0.06503363904335835
tyumen vs yamal
0.0758213082958028
tyumen vs putin
0.031180624158894776

tula vs yamal
0.02292316631166158
tula vs putin
0.08446735731500388

yamal vs putin
0.0914573075919699


ryazan is most similar to Putin speech


In [10]:
def sum_text_vectores_by_parahraph(text):
    """суммируем векторы слов внутри каждого абзаца"""
    paragraph_vectors = []
    for line in text:
        paragraph_vector = np.zeros(len(text[0][0]))
        for word_vec in line:
            paragraph_vector = np.add(paragraph_vector,word_vec)
        paragraph_vectors.append(paragraph_vector)
    return paragraph_vectors

In [11]:
putin_par_vectors = sum_text_vectores_by_parahraph(putin)
rzn_par_vectors = sum_text_vectores_by_parahraph(rzn)
tmn_par_vectors = sum_text_vectores_by_parahraph(tmn)
tula_par_vectors = sum_text_vectores_by_parahraph(tula)
yml_par_vectors = sum_text_vectores_by_parahraph(yml)

In [15]:
def compare_text_vectors_by_paragraphs(text_vect_1, text_vect_2):
    """по очереди сравниваем все абзацы каждого из текстов, возвращаем медиану косинусных близостей"""
    dist_list = []
    for first_ind in range(len(text_vect_1)):
        for second_ind in range(first_ind + 1, len(text_vect_2)):
            dist = distance.cosine(text_vect_1[first_ind],text_vect_2[second_ind])
            dist_list.append(dist)
    dist_list_median = statistics.median(dist_list)
    return dist_list_median

In [ ]:
#Попарно без повторений находим медиану косинусных близостей каждого из полученных векторов. Получаем выводы

In [17]:
par_vectors_dict = {'ryazan':rzn_par_vectors,"tyumen":tmn_par_vectors,"tula": tula_par_vectors, "yamal": yml_par_vectors, "putin":putin_par_vectors}
putin_cosine_distance_dict = {'ryazan':0,"tyumen":0,"tula": 0, "yamal": 0}
ordered_vectors_dict = OrderedDict(par_vectors_dict)
keys = list(ordered_vectors_dict.keys())
values = list(ordered_vectors_dict.values())

for dict_ind in range(len(keys)):
    current_vect = values[dict_ind]
    for compare_ind in range(dict_ind + 1,len(keys)):
        print(keys[dict_ind], "vs", keys[compare_ind])
        compared_vect = values[compare_ind]
        print(compare_text_vectors_by_paragraphs(current_vect,compared_vect))
        
        if(keys[compare_ind] == 'putin'):
            putin_cosine_distance_dict[keys[dict_ind]] = compare_text_vectors_by_paragraphs(current_vect,compared_vect)
    print()
sorted_x = sorted(putin_cosine_distance_dict.items(), key=operator.itemgetter(1))
print(sorted_x[-1][0],"is most similar to Putin speech")

ryazan vs tyumen
0.5728976662871587
ryazan vs tula
0.4949619878988849
ryazan vs yamal
0.48090028602298207
ryazan vs putin
0.5456994849176036

tyumen vs tula
0.5704417922255496
tyumen vs yamal
0.5334847802975864
tyumen vs putin
0.5510098663350491

tula vs yamal
0.48787662434812407
tula vs putin
0.5508491821354387

yamal vs putin
0.5422966409752872


tyumen is most similar to Putin speech
